In [1]:
#싸이파이 arff 파일읽기
from scipy.io import arff
from io import StringIO

import numpy as np
import pandas as pd


## 4번> 별표로 남겨놓기__데이터도 몰겠고, resampling도 몰겠음


# 5번 > 특성추출

In [2]:
from sklearn.feature_selection import SelectKBest,f_classif,chi2

In [3]:
cont_data,cont_meta = arff.loadarff('C:/Users/82105/Desktop/20_2Q_수업/데이터마이닝[세종]/Weka-3-8-4/data/weather.numeric.arff')

print(cont_meta)
cont_data = pd.DataFrame(cont_data)
# display(cont_data)

## 1-2. binary 표현 변경 pd.Categorical 이용

for col in cont_data:
    if col in("temperature","humidity") :
        continue
    else:
        temp = list(map(lambda x: x.decode('UTF-8'),cont_data[col]))
        cont_data[col] = temp
        cont_data[col] = pd.Categorical(cont_data[col])
        
display(cont_data)

Dataset: weather
	outlook's type is nominal, range is ('sunny', 'overcast', 'rainy')
	temperature's type is numeric
	humidity's type is numeric
	windy's type is nominal, range is ('TRUE', 'FALSE')
	play's type is nominal, range is ('yes', 'no')



,outlook,temperature,humidity,windy,play
0,sunny,85.0,85.0,FALSE,no
1,sunny,80.0,90.0,TRUE,no
2,overcast,83.0,86.0,FALSE,yes
3,rainy,70.0,96.0,FALSE,yes
4,rainy,68.0,80.0,FALSE,yes
5,rainy,65.0,70.0,TRUE,no
6,overcast,64.0,65.0,TRUE,yes
7,sunny,72.0,95.0,FALSE,no
8,sunny,69.0,70.0,FALSE,yes
9,rainy,75.0,80.0,FALSE,yes


In [4]:
#날씨 "sunny",0 //overcast",1// "rainy",2
cont_data_categ= cont_data.copy()
cont_data_categ.replace("sunny",0,inplace = True)
cont_data_categ.replace("overcast",1,inplace = True)
cont_data_categ.replace("rainy",2,inplace = True)
#false = 0
cont_data_categ.replace("FALSE",0,inplace = True)
cont_data_categ.replace("TRUE",1,inplace = True)
cont_data_categ.replace("no",0,inplace = True)
cont_data_categ.replace("yes",1,inplace = True)
display(cont_data_categ)

,outlook,temperature,humidity,windy,play
0,0,85.0,85.0,0,0
1,0,80.0,90.0,1,0
2,1,83.0,86.0,0,1
3,2,70.0,96.0,0,1
4,2,68.0,80.0,0,1
5,2,65.0,70.0,1,0
6,1,64.0,65.0,1,1
7,0,72.0,95.0,0,0
8,0,69.0,70.0,0,1
9,2,75.0,80.0,0,1


# A.F검증

In [5]:
select = SelectKBest(score_func = f_classif, k=3)
ab = select.fit_transform(cont_data_categ.iloc[:,0:4],cont_data_categ.iloc[:,4])
select.get_support()

array([ True, False,  True,  True])

## 0,2,3선택
## 후 2개 남김

In [6]:
select = SelectKBest(score_func = f_classif, k=2)
ab = select.fit_transform(cont_data_categ.iloc[:,0:4],cont_data_categ.iloc[:,4])
select.get_support()

array([False, False,  True,  True])

## 2,3선택

# B.chi

In [7]:
select = SelectKBest(score_func = chi2, k=3)
ab = select.fit_transform(cont_data_categ.iloc[:,0:4],cont_data_categ.iloc[:,4])
select.get_support()

array([ True, False,  True,  True])

## 똑같이 0,2,3선택

# C.p검증

In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


In [41]:
full_model = sm.OLS(cont_data_categ.iloc[:,4],cont_data_categ.iloc[:,0:4]) #ordinary least square #최소자승법???
fitted_full_model = full_model.fit()

fitted_full_model.summary()

C:\Users\82105\Anaconda3\lib\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   play   R-squared (uncentered):                   0.699
Model:                            OLS   Adj. R-squared (uncentered):              0.579
Method:                 Least Squares   F-statistic:                              5.817
Date:                Wed, 07 Oct 2020   Prob (F-statistic):                      0.0110
Time:                        22:46:30   Log-Likelihood:                         -8.3582
No. Observations:                  14   AIC:                                      24.72
Df Residuals:                      10   BIC:                                      27.27
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
outlook         0.2040      0.170      1.203      0.257      -0.174       0.582
temperature     0.0245      0.016      1.557      0.151      -0.011       0.059
humidity       -0.0158      0.015     -1.072      0.309      -0.049       0.017
windy          -0.1926      0.274     -0.702      0.499      -0.804       0.419
==============================================================================
Omnibus:                        1.835   Durbin-Watson:                   1.717
Prob(Omnibus):                  0.400   Jarque-Bera (JB):                1.077
Skew:                          -0.354   Prob(JB):                        0.584
Kurtosis:                       1.840   Cond. No.                         218.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
cont_data_categ.iloc[:,4]

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: play, dtype: category
Categories (2, int64): [0, 1]

In [13]:
#GLM
full_model = sm.GLM(cont_data_categ.iloc[:,4],cont_data_categ.iloc[:,0:4],
                   family = sm.families.Binomial()) #ordinary least square #최소자승법???
fitted_full_model = full_model.fit()

fitted_full_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   play   No. Observations:                   14
Model:                            GLM   Df Residuals:                       10
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -7.6224
Date:                Thu, 08 Oct 2020   Deviance:                       15.245
Time:                        12:15:11   Pearson chi2:                     12.9
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
outlook         0.9288      0.799      1.163      0.245      -0.637       2.494
temperature     0.0966      0.074      1.302      0.193      -0.049       0.242
humidity       -0.0837      0.068     -1.229      0.219      -0.217       0.050
windy          -1.1640      1.241     -0.938      0.348      -3.597       1.269
===============================================================================
"""

In [10]:
#로지스틱회귀분석
full_model = sm.Logit(cont_data_categ.iloc[:,4],cont_data_categ.iloc[:,0:4]) #ordinary least square #최소자승법???
fitted_full_model = full_model.fit()

fitted_full_model.summary()

Optimization terminated successfully.
         Current function value: 0.544455
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   play   No. Observations:                   14
Model:                          Logit   Df Residuals:                       10
Method:                           MLE   Df Model:                            3
Date:                Thu, 08 Oct 2020   Pseudo R-squ.:                  0.1646
Time:                        12:09:52   Log-Likelihood:                -7.6224
converged:                       True   LL-Null:                       -9.1246
Covariance Type:            nonrobust   LLR p-value:                    0.3909
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
outlook         0.9288      0.799      1.163      0.245      -0.637       2.494
temperature     0.0966      0.074      1.302      0.193      -0.049       0.242
humidity       -0.0837      0.068     -1.229      0.219      -0.217       0.050
windy          -1.1640      1.241     -0.938      0.348      -3.597       1.269
===============================================================================
"""

## d.다중공선성(VIF)확인

In [17]:
cont_data_categ.values[:,0:3]

array([[ 0., 85., 85.],
       [ 0., 80., 90.],
       [ 1., 83., 86.],
       [ 2., 70., 96.],
       [ 2., 68., 80.],
       [ 2., 65., 70.],
       [ 1., 64., 65.],
       [ 0., 72., 95.],
       [ 0., 69., 70.],
       [ 2., 75., 80.],
       [ 0., 75., 70.],
       [ 1., 72., 90.],
       [ 1., 81., 75.],
       [ 2., 71., 91.]])

In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(cont_data_categ.values[:,0:4], i)
                     for i in range(cont_data_categ.shape[1]-1)] #shape 1인덱스가 컬럼수
vif["features"] = cont_data_categ.columns[0:4]
vif.sort_values("VIF Factor",ascending= False)

,VIF Factor,features
2,75.930537,humidity
1,69.689299,temperature
0,2.551763,outlook
3,1.670630,windy


## 6번 J48은 별표

# 7. 나이브 베이즈 분류

In [50]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

In [47]:
cont_data,cont_meta = arff.loadarff('C:/Users/82105/Desktop/20_2Q_수업/데이터마이닝[세종]/Weka-3-8-4/data/contact-lenses.arff')

print(cont_meta)
cont_data = pd.DataFrame(cont_data)
# display(cont_data)

## 1-2. binary 표현 변경 pd.Categorical 이용

for col in cont_data:
    if col == "Result":
        temp = list(map(lambda x: int(x.decode('UTF-8')),cont_data[col]))
        cont_data[col] = temp
    else:
        temp = list(map(lambda x: x.decode('UTF-8'),cont_data[col]))
        cont_data[col] = temp
        cont_data[col] = pd.Categorical(cont_data[col])
        
display(cont_data)

Dataset: contact-lenses
	age's type is nominal, range is ('young', 'pre-presbyopic', 'presbyopic')
	spectacle-prescrip's type is nominal, range is ('myope', 'hypermetrope')
	astigmatism's type is nominal, range is ('no', 'yes')
	tear-prod-rate's type is nominal, range is ('reduced', 'normal')
	contact-lenses's type is nominal, range is ('soft', 'hard', 'none')



,age,spectacle-prescrip,astigmatism,tear-prod-rate,contact-lenses
0,young,myope,no,reduced,none
1,young,myope,no,normal,soft
2,young,myope,yes,reduced,none
3,young,myope,yes,normal,hard
4,young,hypermetrope,no,reduced,none
5,young,hypermetrope,no,normal,soft
6,young,hypermetrope,yes,reduced,none
7,young,hypermetrope,yes,normal,hard
8,pre-presbyopic,myope,no,reduced,none
9,pre-presbyopic,myope,no,normal,soft


In [53]:
## label encoding 
le = LabelEncoder()
temp_encoded = cont_data.copy()
for col in cont_data: #라벨 인코딩!!
    temp_encoded[col]=le.fit_transform(cont_data[col])
display(temp_encoded)

,age,spectacle-prescrip,astigmatism,tear-prod-rate,contact-lenses
0,2,1,0,1,1
1,2,1,0,0,2
2,2,1,1,1,1
3,2,1,1,0,0
4,2,0,0,1,1
5,2,0,0,0,2
6,2,0,1,1,1
7,2,0,1,0,0
8,0,1,0,1,1
9,0,1,0,0,2


## soft 2 none 1 hard 0

# 0~ 19까지 학습 후 20~23 예측진행

In [78]:
#Create a Gaussian Classifier
model = GaussianNB()

features = temp_encoded.iloc[:,0:4]
label = temp_encoded.iloc[:,4]

# Train the model using the training sets
model.fit(features.iloc[:20,:],label.iloc[:20])

GaussianNB()

In [80]:
#Predict Output
# presbyopic hypermetrope	yes	reduced   (1	0	1	1)
predicted= model.predict(features.iloc[20:,:])
print("Predicted Value:", predicted) # 1: none 예측함
print("실제 :",list(label[20:]))

Predicted Value: [1 2 1 0]
실제 : [1, 2, 1, 1]


# 8.• IB1, IBk:  ==K최근접

In [67]:
# 최근접
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors =3)

clf.fit(features.iloc[:20,:],label.iloc[:20])

KNeighborsClassifier(n_neighbors=3)

In [76]:
print("예측 : {}".format(clf.predict(features.iloc[20:,:])))
print("정답 : ")
print(list(label.iloc[20:]))

예측 : [1 2 1 0]
정답 : 
[1, 2, 1, 1]


In [82]:
# 1최근접
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors =1)

clf.fit(features.iloc[:20,:],label.iloc[:20])
print("예측 : {}".format(clf.predict(features.iloc[20:,:])))
print("정답 : ")
print(list(label.iloc[20:]))

예측 : [1 2 1 0]
정답 : 
[1, 2, 1, 1]


# 9번. 연관 분석

In [21]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


In [20]:
cont_data,cont_meta = arff.loadarff('C:/Users/82105/Desktop/20_2Q_수업/데이터마이닝[세종]/Weka-3-8-4/data/weather.nominal.arff')

print(cont_meta)
cont_data = pd.DataFrame(cont_data)
# display(cont_data)

## 1-2. binary 표현 변경 pd.Categorical 이용

for col in cont_data:
    if col == "Result":
        temp = list(map(lambda x: int(x.decode('UTF-8')),cont_data[col]))
        cont_data[col] = temp
    else:
        temp = list(map(lambda x: x.decode('UTF-8'),cont_data[col]))
        cont_data[col] = temp
        cont_data[col] = pd.Categorical(cont_data[col])
        
display(cont_data)

Dataset: weather.symbolic
	outlook's type is nominal, range is ('sunny', 'overcast', 'rainy')
	temperature's type is nominal, range is ('hot', 'mild', 'cool')
	humidity's type is nominal, range is ('high', 'normal')
	windy's type is nominal, range is ('TRUE', 'FALSE')
	play's type is nominal, range is ('yes', 'no')



,outlook,temperature,humidity,windy,play
0,sunny,hot,high,FALSE,no
1,sunny,hot,high,TRUE,no
2,overcast,hot,high,FALSE,yes
3,rainy,mild,high,FALSE,yes
4,rainy,cool,normal,FALSE,yes
5,rainy,cool,normal,TRUE,no
6,overcast,cool,normal,TRUE,yes
7,sunny,mild,high,FALSE,no
8,sunny,cool,normal,FALSE,yes
9,rainy,mild,normal,FALSE,yes


In [33]:
te= TransactionEncoder()
tr_ary = te.fit(cont_data.values).transform(cont_data.values)
df  = pd.DataFrame(tr_ary,columns=te.columns_)
df

,FALSE,TRUE,cool,high,hot,mild,no,normal,overcast,rainy,sunny,yes
0,True,False,False,True,True,False,True,False,False,False,True,False
1,False,True,False,True,True,False,True,False,False,False,True,False
2,True,False,False,True,True,False,False,False,True,False,False,True
3,True,False,False,True,False,True,False,False,False,True,False,True
4,True,False,True,False,False,False,False,True,False,True,False,True
5,False,True,True,False,False,False,True,True,False,True,False,False
6,False,True,True,False,False,False,False,True,True,False,False,True
7,True,False,False,True,False,True,True,False,False,False,True,False
8,True,False,True,False,False,False,False,True,False,False,True,True
9,True,False,False,False,False,True,False,True,False,True,False,True


In [36]:
frequent_itemsets = apriori(df, min_support=0.3, use_colnames=True)
frequent_itemsets 

,support,itemsets
0,0.571429,(FALSE)
1,0.428571,(TRUE)
2,0.500000,(high)
3,0.428571,(mild)
4,0.357143,(no)
5,0.500000,(normal)
6,0.357143,(rainy)
7,0.357143,(sunny)
8,0.642857,(yes)
9,0.428571,"(FALSE, yes)"
